# Amazon recommender

## Data import + cleaning

In [1]:
#libraries
import pandas as pd, numpy as np, matplotlib.pyplot as plt, \
seaborn as sns

### Training data

In [16]:
#fetch data for TRAIN
orders = pd.read_csv('../data/01-Jan-2008_to_18-May-2021.csv')

In [17]:
#peek at data for TRAIN
orders.head(2)

,Order Date,Order ID,Title,Category,ASIN/ISBN,UNSPSC Code,Website,Release Date,Condition,Seller,...,Carrier Name & Tracking Number,Item Subtotal,Item Subtotal Tax,Item Total,Tax Exemption Applied,Tax Exemption Type,Exemption Opt-Out,Buyer Name,Currency,Group Name
0,12/15/08,002-9208212-4089021,NaN,NaN,B0006HGNJ4,NaN,Amazon.com,NaN,NaN,Watch Values,...,USPS(9101805213907299379333),$24.95,$0.00,$24.95,NaN,NaN,NaN,Veronica,USD,NaN
1,03/23/09,058-3723913-1124538,The Wall,ABIS_MUSIC,B000006TRV,55111512.0,Amazon.com,1994-01-01T00:00,new,megahitrecords,...,NaN,$15.24,$0.00,$15.24,NaN,NaN,NaN,Veronica,USD,NaN


In [6]:
#understand prospective features
orders.columns

Index(['Order Date', 'Order ID', 'Title', 'Category', 'ASIN/ISBN',
       'UNSPSC Code', 'Website', 'Release Date', 'Condition', 'Seller',
       'Seller Credentials', 'List Price Per Unit', 'Purchase Price Per Unit',
       'Quantity', 'Payment Instrument Type', 'Purchase Order Number',
       'PO Line Number', 'Ordering Customer Email', 'Shipment Date',
       'Shipping Address Name', 'Shipping Address Street 1',
       'Shipping Address Street 2', 'Shipping Address City',
       'Shipping Address State', 'Shipping Address Zip', 'Order Status',
       'Carrier Name & Tracking Number', 'Item Subtotal', 'Item Subtotal Tax',
       'Item Total', 'Tax Exemption Applied', 'Tax Exemption Type',
       'Exemption Opt-Out', 'Buyer Name', 'Currency', 'Group Name'],
      dtype='object')

In [7]:
#understand size of data
orders.shape

(2242, 36)

In [8]:
#data types we are dealing with and presence of nulls
orders.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2242 entries, 0 to 2241
Data columns (total 36 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   Order Date                      2242 non-null   object 
 1   Order ID                        2242 non-null   object 
 2   Title                           2163 non-null   object 
 3   Category                        2163 non-null   object 
 4   ASIN/ISBN                       2242 non-null   object 
 5   UNSPSC Code                     2163 non-null   float64
 6   Website                         2242 non-null   object 
 7   Release Date                    77 non-null     object 
 8   Condition                       2241 non-null   object 
 9   Seller                          2241 non-null   object 
 10  Seller Credentials              5 non-null      object 
 11  List Price Per Unit             2242 non-null   object 
 12  Purchase Price Per Unit         22

In [ ]:
#convert code to object


In [18]:
#descriptive stats
orders.describe().round(4).T

,count,mean,std,min,25%,50%,75%,max
UNSPSC Code,2163.0,4.909079e+07,9.088133e+06,10111300.0,50000000.0,50202300.0,53102500.5,78130000.0
Quantity,2242.0,1.102600e+00,4.331000e-01,0.0,1.0,1.0,1.0,7.0
Purchase Order Number,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
PO Line Number,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Shipping Address Street 2,641.0,5.040000e+02,0.000000e+00,504.0,504.0,504.0,504.0,504.0
Tax Exemption Type,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Group Name,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Testing data

In [6]:
#import data for TEST
#from https://stackoverflow.com/questions/9652832/how-to-load-a-tsv-file-into-a-pandas-dataframe
#read_csv('path_to_file', sep='\t')
products = pd.read_csv(
    '../data/marketing_sample_for_amazon_com-amazon_product_data__20200401_20200630__30k_data.tsv',
sep='\t')

In [19]:
#peek
products.head(2)

,Uniq Id,Crawl Timestamp,Dataset Origin,Product Id,Product Barcode,Product Company Type Source,Product Brand Source,Product Brand Normalised Source,Product Name Source,Match Rank,...,Product Currency,Product Available Inventory,Product Image Url,Product Model Number,Product Tags,Product Contents,Product Rating,Product Reviews Count,Bsr,Joining Key
0,690e47123b04e15df768a9f54f1654e8,2020-05-13 11:02:04 +0000,NaN,ed177532df52b8b2fdba6cff731b9d00,NaN,Competitor,NaN,NaN,NaN,NaN,...,USD,999999999,https://images-na.ssl-images-amazon.com/images...,NaN,100% Linen Table Runner 14x90 Tailored with Mi...,100% Pure Linen Imported 100% Pure Linen table...,3.5,4,"#83,321 in Kitchen & Dining (See Top 100 in Ki...",78af1f66f1e365647484688360621e5e
1,b02a91cfdae2c5596b568e771b402176,2020-06-27 15:31:29 +0000,NaN,8756f065784790599a218174bc932395,UPC: 799460760240,Competitor,NaN,NaN,NaN,NaN,...,USD,999999999,https://images-na.ssl-images-amazon.com/images...,NL-F-5-2,2 pcs/set LONG Home Sauna Spa Exfoliating Nylo...,The nylon exfoliating cloth is the best bath p...,0.0,0,"#175,862 in Beauty & Personal Care (See Top 10...",83e8d81095fe218d5bb580194669e0c4


In [8]:
products.columns

Index(['Uniq Id', 'Crawl Timestamp', 'Dataset Origin', 'Product Id',
       'Product Barcode', 'Product Company Type Source',
       'Product Brand Source', 'Product Brand Normalised Source',
       'Product Name Source', 'Match Rank', 'Match Score', 'Match Type',
       'Retailer', 'Product Category', 'Product Brand', 'Product Name',
       'Product Price', 'Sku', 'Upc', 'Product Url', 'Market',
       'Product Description', 'Product Currency',
       'Product Available Inventory', 'Product Image Url',
       'Product Model Number', 'Product Tags', 'Product Contents',
       'Product Rating', 'Product Reviews Count', 'Bsr', 'Joining Key'],
      dtype='object')

In [11]:
products.shape

(29984, 32)

In [13]:
products.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29984 entries, 0 to 29983
Data columns (total 32 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   Uniq Id                          29984 non-null  object 
 1   Crawl Timestamp                  29984 non-null  object 
 2   Dataset Origin                   0 non-null      float64
 3   Product Id                       29984 non-null  object 
 4   Product Barcode                  6291 non-null   object 
 5   Product Company Type Source      29984 non-null  object 
 6   Product Brand Source             71 non-null     object 
 7   Product Brand Normalised Source  71 non-null     object 
 8   Product Name Source              71 non-null     object 
 9   Match Rank                       0 non-null      float64
 10  Match Score                      0 non-null      float64
 11  Match Type                       0 non-null      float64
 12  Retailer          

In [15]:
products.describe().T

,count,mean,std,min,25%,50%,75%,max
Dataset Origin,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Match Rank,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Match Score,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Match Type,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Product Price,23553.0,4.128903e+01,1.211586e+02,1.000000e-02,1.129000e+01,18.5,3.499000e+01,7400.0
Product Available Inventory,29984.0,9.099964e+08,2.681509e+08,1.111111e+08,1.000000e+09,999999999.0,1.000000e+09,999999999.0
Product Rating,29984.0,3.122762e+00,1.978487e+00,0.000000e+00,0.000000e+00,4.1,4.600000e+00,5.0
Product Reviews Count,29984.0,5.356103e+01,3.419812e+02,0.000000e+00,0.000000e+00,3.0,2.000000e+01,24231.0


## EDA

In [20]:
#let's analyze the categories both in train and test
orders['Category'].value_counts()

GROCERY                   170
VEGETABLE                 132
FRUIT                      83
SKIN_CLEANING_AGENT        81
INCONTINENCE_PROTECTOR     66
                         ... 
MANUAL_SHAVING_RAZOR        1
BAG                         1
AREA_DEODORIZER             1
DECORATIVE_RIBBON_TRIM      1
BUILDING_MATERIAL           1
Name: Category, Length: 305, dtype: int64

## Modeling & Recommender system

## Evaluation

## Next steps